In [ ]:
from tqdm import tqdm_notebook
import os
import json
os.chdir('../repos/darknet/')

In [ ]:
!chmod +x ./darknet

In [ ]:
!./darknet detector map data/obj.data cfg/yolo-obj.cfg backup/yolo-obj_last.weights

In [ ]:
!./darknet detector test data/obj.data cfg/yolo-obj.cfg backup/yolo-obj_last.weights -ext_output -dont_show -out result.json < data/submission_test.txt > test_output_capture.txt

In [ ]:
with open('test_output_capture.txt') as out:
    data = out.read()

In [ ]:
#submission code
mapping = {
  'paragraph':0,
  'dropdown':1,
  'checkbox':2,
  'radiobutton':3,
  'rating':4,
  'toggle':5,
  'textarea':6,
  'datepicker':7,
  'stepperinput':8,
  'slider':9,
  'video':10,
  'label':11,
  'table':12,
  'list':13,
  'header':14,
  'button':15,
  'image':16,
  'linebreak':17,
  'container':18,
  'link':19,
  'textinput':20
}
coco = []
img_id = ''
for line in tqdm_notebook(data.split('\n')):
    if 'Enter Image Path:' in line:
        img_path = line.split(' ')[3].split(':')[0]
        img_id = img_path.split('/')[-1].split('.')[0]
    elif 'left_x:' in line:
        strstr = line
        log = strstr.split('\t')
        class_name = log[0].split(':')[0]
        score = log[0].split(':')[1].split('%')[0]
        left_x = int(strstr.split('left_x:')[1].split('top_y:')[0])
        top_y = int(strstr.split('top_y:')[1].split('width:')[0])
        width = int(strstr.split('width:')[1].split('height:')[0])
        height = int(strstr.split('height:')[1].split(')')[0])

        obj = {
            'image_id': img_id,
            'category_id': mapping[class_name],
            'bbox': [left_x, top_y, width, height],
            'score': int(score)/100
        }
        coco.append(obj)

In [ ]:
with open('../../results/yolo_predictions.json','wt') as out:
    json.dump(coco,out)